In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

using Markdown

# Import some libraries from the JDP project
using JDP.BugRefs
using JDP.Tracker
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository
using JDP.Functional

# Deleting OpenQA comments

This workbook demonstrates deleting OpenQA comments programatically

## Refreshing/Getting the data

We can choose to refresh the comments based on some arbitrary filter. Comments are refreshed on a per job basis.

In [ ]:
build = "188.1"
    
OpenQA.refresh_comments(job -> job.vars["BUILD"] == build, "osd")

Then we get all the jobs

In [ ]:
alljobs = Repository.fetch(OpenQA.JobResult, Vector, "osd")
md"We now have $(length(allres)) job results"

## Choosing which comments to delete

Now we have a vector of `JobResult` structs

In [ ]:
@doc OpenQA.JobResult

We can filter this in a number of stages, narrowing it down to just the jobs containing comments we care about

In [ ]:
jobs = filter(alljobs) do job
    job.vars["BUILD"] == build
end

In [ ]:
jobs = filter(jobs) do job
    !isempty(job.comments)
end

Before doing further filtering we can project the comments into a new, flattened, array. This just makes the next stage of filtering easier because we are not dealing with a nested structure.

In [ ]:
coms = (((job.id, comment) for comment in job.comments) for job in jobs) |> 
    Iterators.flatten |>
    collect
    

Then we find filter the comments to only those starting with `label:linked` and display them to do a sanity check.

In [ ]:
todel = filter(coms) do (id, com)
    startswith(com.text, "label:linked")
end

for (id, com) in todel
    display(md"## Comment $(com.id) on $id by $(com.author)")
    display(Markdown.parse(com.text))
end

## Deleting the comments

Finally we actually delete them.

In [ ]:
oqa = get_tracker(load_trackers(), "osd")
ses = Tracker.ensure_login!(oqa)

for (id, com) in todel
    @info "Deleting $(com.id)"
    OpenQA.delete_job_comment(ses, id, com.id)
end